In [1]:
import pandas as pd
import numpy as np

import plotly.express as px

In [2]:
df = pd.read_csv('/Users/genie/data/covid-19/owid-covid-data.csv')
df = df[(df.iso_code.notnull()) & ~(df.iso_code.isin(['OWID_KOS','OWID_WRL']))]
df = df.sort_values(by=['date'])
df['new_date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%U')

# df['new_cases'] = df.mask(df['new_cases'] < 0, np.nan)
df['new_cases'] = df.apply(lambda x:x['new_cases'] if x['new_cases'] >=0 else np.nan, axis=1)

df_latest = df[df.date == df.date.max()]

In [3]:
df_latest.shape

(207, 35)

In [4]:
fig = px.choropleth(df, locations="iso_code",
                    color="new_cases",
                    hover_name="location",
                    animation_frame="date",
                    title = "Daily new COVID cases",
                    color_continuous_scale=px.colors.sequential.OrRd)

fig["layout"].pop("updatemenus")
# fig.show()
fig.write_html("file.html")